In [1]:
import numpy as np 
import pandas as pd

from helpers import Helper
helper = Helper()

In [2]:
inventory = pd.read_csv('Inventory_Query_US_CA.csv')
prod_info = pd.read_excel('Product_Info.xlsx')
site_info = pd.read_excel('Site_Info.xlsx')
target = pd.read_excel('Mar20_23_Inventory_Clean.xlsx', sheet_name='March 20 Inventory Clean')
prod2 = pd.read_csv('prod2.csv')
result = pd.read_excel('inventory_output.xlsx')

In [3]:
inventory = helper.remove_whitespace_from_column_names(inventory)
prod_info = helper.remove_whitespace_from_column_names(prod_info)
site_info = helper.remove_whitespace_from_column_names(site_info)
target = helper.remove_whitespace_from_column_names(target)

### Step 0

Only look at accessories and hardware data in inventory.

In [4]:
inventory = inventory[(inventory['list_item_name'] == 'CF: Accessory') | (inventory['list_item_name'] == 'CF: Hardware')]


### Step 1

Add `Site Type` info.

In [5]:
step1_st = inventory.merge(right=site_info, on='loc', how='left')
step1_st = helper.move_column_after(step1_st, 'Site Type', 'loc')

### Step 2

Add `Product Group` and `Product Family` info.

In [6]:
step2_pgpf = step1_st.merge(right=prod_info, on='item', how='left')
step2_pgpf = helper.move_column_after(step2_pgpf, 'Product Group', 'item')
step2_pgpf = helper.move_column_after(step2_pgpf, 'Product Family', 'Product Group')

### Step 3

Add `Sellable/Non Sellable` column.

In [7]:
step3_sell = step2_pgpf
step3_sell['Sellable/Non Sellable'] = step3_sell['bucket'].apply(lambda x: 'Sellable' if x == 'Sellable' or x == 'In Transit' else 'NonSellable')
step3_sell = helper.move_column_after(step3_sell, 'Sellable/Non Sellable', 'bucket')

### Step 4

Compute `Pallets` column.

In [8]:
step4_pallets = step3_sell
step4_pallets['Pallets'] = step4_pallets['qty']/step4_pallets['Units/Pallet']

### Step 5

Add multiplier info.

In [9]:
step5_multiplier = step4_pallets
step5_multiplier['multiplier'] = step5_multiplier['Sellable/Non Sellable'].apply(lambda x: 2 if x == 'NonSellable' else 1)
step5_multiplier.loc[step5_multiplier["item"].isin(prod2["item"]), 'multiplier'] = 2

### Step 6

Compute pallet positions.

In [10]:
step6_positions = step5_multiplier
step6_positions['Pallet Positions'] = np.ceil(step6_positions['Pallets']*step6_positions['multiplier'])

### Step 7

Drop extra columns.

In [11]:
result = step6_positions
result = result.drop(columns=['warehouse_code', 'multiplier'])

### Export Result

In [12]:
result.to_excel('inventory_output.xlsx', index=False)